In [1]:
import tensorflow as tf
from tensorflow import keras
from keras import datasets, layers, models, regularizers
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import precision_score, recall_score
import numpy as np
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from keras.models import load_model
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.callbacks import LearningRateScheduler, EarlyStopping, ModelCheckpoint, TensorBoard
import random
import os
import shutil

2024-01-10 13:27:37.075290: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-10 13:27:37.075454: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-10 13:27:37.393292: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-10 13:27:38.046248: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-10 13:27:42.206745: W tensorflow/compiler/tf2

In [14]:
dataset_dir = '/workspaces/LungsXray-FP/data/raw/BINARY_COVID'

In [15]:
image_size = (150, 150)
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    rotation_range=20,  # Agregar rotación en grados
    validation_split=0.2    
)

train_generator = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary',  
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'
)


Found 2488 images belonging to 2 classes.
Found 622 images belonging to 2 classes.


In [16]:
num_classes = len(train_generator.class_indices)

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(image_size[0], image_size[1], 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),  # Agregar capa de dropout para prevenir el sobreajuste
    tf.keras.layers.Dense(1, activation='sigmoid')  # Clasificación binaria, usar 'binary_crossentropy'
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [17]:
epochs = 10

history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator
)

Epoch 1/10
78/78 [==============================] - 143s 1s/step - loss: 0.4762 - accuracy: 0.7721 - val_loss: 0.4502 - val_accuracy: 0.8248
Epoch 2/10
78/78 [==============================] - 97s 1s/step - loss: 0.3018 - accuracy: 0.8834 - val_loss: 0.4033 - val_accuracy: 0.8441
Epoch 3/10
78/78 [==============================] - 100s 1s/step - loss: 0.2533 - accuracy: 0.9084 - val_loss: 0.3655 - val_accuracy: 0.8826
Epoch 4/10
78/78 [==============================] - 97s 1s/step - loss: 0.2011 - accuracy: 0.9297 - val_loss: 0.3699 - val_accuracy: 0.8666
Epoch 5/10
78/78 [==============================] - 96s 1s/step - loss: 0.2331 - accuracy: 0.9035 - val_loss: 0.3761 - val_accuracy: 0.8473
Epoch 6/10
78/78 [==============================] - 97s 1s/step - loss: 0.2190 - accuracy: 0.9228 - val_loss: 0.3867 - val_accuracy: 0.8617
Epoch 7/10
78/78 [==============================] - 97s 1s/step - loss: 0.1690 - accuracy: 0.9353 - val_loss: 0.5027 - val_accuracy: 0.8762
Epoch 8/10
78/78 [

In [18]:
test_loss, test_acc = model.evaluate(validation_generator, verbose=2)
print('\nExactitud en el conjunto de validación:', test_acc)


20/20 - 20s - loss: 0.3436 - accuracy: 0.8810 - 20s/epoch - 998ms/step

Exactitud en el conjunto de validación: 0.881028950214386


In [ ]:
model.save('modelo_binary_COVID.keras')

In [ ]:
model = load_model('modelo_binary_COVID.keras')  # Ajusta el nombre de tu modelo
# Preprocesamiento de datos para el conjunto de prueba
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    dataset_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',  # Ajusta si tu modelo espera etiquetas categóricas
    shuffle=False
)

In [ ]:
# Obtener las predicciones del modelo para el conjunto de prueba
predictions = model.predict(test_generator)
# Convertir las predicciones a clases (si es necesario)
predicted_classes = np.argmax(predictions, axis=1)
# Obtener las etiquetas verdaderas
true_classes = test_generator.classes
# Calcular la precisión y el recall por clase
classification_rep = classification_report(true_classes, predicted_classes)
accuracy = accuracy_score(true_classes, predicted_classes)
conf_matrix = confusion_matrix(true_classes, predicted_classes)
print("Informe de clasificación:")
print(classification_rep)
print("Exactitud general:")
print(accuracy)
print("Matriz de confusión:")
print(conf_matrix)